In [ ]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
train = pd.read_csv('ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
train = train.dropna()

In [ ]:
train

,category,text
0,None-of-the-above,enaku unmaikum aluha wantu thirunangaigal thei...
1,Transphobic,SUPERSTAR VIJAY dai arivuketta polu ithu thapp...
2,None-of-the-above,Ugka smile cute a iruku😊😊...
3,None-of-the-above,Anna i am waiting na 🥰🥰🥰
4,None-of-the-above,Yanda tamilnadu la evvalavo pirachana iruku at...
...,...,...
5943,None-of-the-above,Noq day having no caste no religious certificate
5944,None-of-the-above,Mimicry escape aitaru bro. 😂
5945,None-of-the-above,Rajesh R u age u from
5946,Counter-speech,Bro unga videos nallarukku but don't do prank ...


In [ ]:
labels = train.category.unique()
labels

array(['None-of-the-above', 'Transphobic', 'Counter-speech', 'Misandry',
       'Homophobia', 'Hope-Speech', 'Xenophobia', 'Misogyny'],
      dtype=object)

In [ ]:
train = pd.read_csv('ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
train = train.dropna()

In [ ]:
test = pd.read_csv('ta-en-misogyny-dev.csv',sep='\t', names = ["category", "text"])
test = test.dropna()

In [ ]:
import string
def removepunctuation(text):
    clean="".join([i for i in text if i not in string.punctuation])
    return clean

import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def tolower(text):
  return text.lower()

label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga']) 
stopwords_list.extend(['enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha']) 

label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

def removestopwords(text):
  clean = " ".join([word for word in text.split() if word not in stopwords_list])
  return clean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess(df):
  df['text']= df['text'].apply(lambda x:tolower(x))
  df['text']= df['text'].apply(lambda x:removepunctuation(x))
  df['text']= df['text'].apply(lambda x:deEmojify(x))
  df['text']= df['text'].apply(lambda x:removestopwords(x))
  df["category"] = df["category"].map(label)
  return df

In [ ]:
train = preprocess(train)
train

,category,text
0,0,enaku unmaikum aluha wantu thirunangaigal thei...
1,1,superstar vijay arivuketta polu ithu thappu pa...
2,0,ugka smile cute
3,0,anna waiting 🥰🥰🥰
4,0,yanda tamilnadu evvalavo pirachana athalam vit...
...,...,...
5943,0,noq day caste religious certificate
5944,0,mimicry escape aitaru
5945,0,rajesh age
5946,2,videos nallarukku dont prank eena varavanga ep...


In [ ]:
test = preprocess(test)

In [ ]:
def tokenize(text):
  return text.split()


train['token']= train['text'].apply(lambda x:tokenize(x))
test['token']= test['text'].apply(lambda x:tokenize(x))

In [ ]:
train

,category,text,token
0,0,enaku unmaikum aluha wantu thirunangaigal thei...,"[enaku, unmaikum, aluha, wantu, thirunangaigal..."
1,1,superstar vijay arivuketta polu ithu thappu pa...,"[superstar, vijay, arivuketta, polu, ithu, tha..."
2,0,ugka smile cute,"[ugka, smile, cute]"
3,0,anna waiting 🥰🥰🥰,"[anna, waiting, 🥰🥰🥰]"
4,0,yanda tamilnadu evvalavo pirachana athalam vit...,"[yanda, tamilnadu, evvalavo, pirachana, athala..."
...,...,...,...
5943,0,noq day caste religious certificate,"[noq, day, caste, religious, certificate]"
5944,0,mimicry escape aitaru,"[mimicry, escape, aitaru]"
5945,0,rajesh age,"[rajesh, age]"
5946,2,videos nallarukku dont prank eena varavanga ep...,"[videos, nallarukku, dont, prank, eena, varava..."


In [ ]:
from collections import Counter
x = train.groupby("category")["text"].apply(lambda x: Counter(" ".join(x).split()).most_common(50))
x[6]

[('china', 120),
 ('virus', 19),
 ('tamil', 17),
 ('corona', 14),
 ('jaadhi', 14),
 ('indha', 13),
 ('sappa', 13),
 ('thulukka', 13),
 ('thayvidiyaa', 13),
 ('avan', 12),
 ('india', 11),
 ('world', 11),
 ('first', 11),
 ('antha', 11),
 ('vera', 10),
 ('mela', 10),
 ('iruka', 10),
 ('peru', 10),
 ('karan', 10),
 ('irundhu', 10),
 ('innum', 9),
 ('america', 8),
 ('pottu', 8),
 ('chinna', 8),
 ('vittu', 8),
 ('ulaga', 8),
 ('nalla', 8),
 ('kooti', 8),
 ('pola', 8),
 ('bomb', 8),
 ('sari', 8),
 ('chaina', 7),
 ('ya', 7),
 ('summa', 7),
 ('enga', 7),
 ('thaan', 7),
 ('vida', 7),
 ('vanthu', 7),
 ('ena', 7),
 ('yen', 7),
 ('kooda', 7),
 ('mattum', 7),
 ('ellam', 7),
 ('ithu', 6),
 ('matum', 6),
 ('daa', 6),
 ('dey', 6),
 ('poda', 6),
 ('vandhu', 6),
 ('idhu', 6)]

In [ ]:
processed = [key for groups in x for key, value in groups]


In [ ]:
uniqueset = set(processed)
processed = list(uniqueset) 

In [ ]:
processed

['unakku',
 'lam',
 'thevidiya',
 'iruka',
 'kuda',
 'stop',
 'thappu',
 'lesbian',
 'age',
 'nanba',
 'feel',
 'adha',
 'support',
 'neega',
 'ya',
 'vandhu',
 'nai',
 'namma',
 'aan',
 'sappa',
 'bomb',
 'help',
 'marriage',
 'train',
 'kevalamana',
 'pa',
 'pavam',
 'super',
 'ponna',
 'poda',
 'ena',
 'homosexual',
 'keep',
 'culture',
 'vera',
 'ponnu',
 'thiru',
 '2',
 'antha',
 'first',
 'mooditu',
 'anbu',
 'avala',
 'hiv',
 'hraja',
 'channel',
 'kita',
 'nalla',
 'pon',
 'kooti',
 'pondatti',
 'panni',
 'avangala',
 'pundaya',
 'pola',
 'irundhu',
 'arivu',
 'kalyanam',
 'entha',
 'atha',
 'ippadi',
 'per',
 'hindu',
 'natarajan',
 'mutta',
 'anitha',
 'yen',
 'dey',
 'storm',
 'varum',
 'part',
 'pasanga',
 'good',
 'paiya',
 'dowry',
 'andha',
 'sex',
 'prank',
 'idhu',
 'amma',
 'onnu',
 'world',
 'china',
 'ungomma',
 'illai',
 'thulukka',
 'ungaluku',
 'madhiri',
 'kumar',
 'sila',
 'best',
 'koodhi',
 'ithu',
 'manasu',
 'avan',
 'epdi',
 'ivan',
 'naan',
 'appa',
 'kad

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',vocabulary=processed)
X_train = tfidfvectorizer.fit_transform(train["text"])
X_test = tfidfvectorizer.fit_transform(test["text"])

In [ ]:
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 3.5 MB 33.5 MB/s 
     |████████████████████████████████| 6.8 MB 33.4 MB/s 
     |████████████████████████████████| 1.2 MB 33.3 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 596 kB 44.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=4c85111a23078642be8c49f7c6268d93d053dbfeb000f0d2a0cd65e0d9e042f0
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/LaBSE')
embedding_train = embedding_model.encode(train["text"].tolist())
embedding_train

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

array([[ 0.03095993, -0.04980514, -0.00067633, ..., -0.04532452,
        -0.07391862, -0.0073102 ],
       [ 0.03705017, -0.01570936, -0.01769593, ...,  0.02685197,
         0.01320322, -0.06840485],
       [ 0.02942532, -0.05950559,  0.01268851, ..., -0.04530559,
        -0.01561758, -0.02738027],
       ...,
       [-0.00462898, -0.05392351, -0.02290568, ..., -0.03905746,
        -0.03931322, -0.01579572],
       [ 0.0108578 , -0.017245  , -0.01349183, ..., -0.07366004,
        -0.02297129,  0.01882049],
       [ 0.08405842, -0.04019969,  0.01693769, ..., -0.04509797,
        -0.04731593,  0.01547591]], dtype=float32)

In [ ]:
embedding_test = embedding_model.encode(test["text"].tolist())
embedding_test

array([[ 0.01120974, -0.03313988,  0.00473972, ..., -0.0256553 ,
        -0.05266365, -0.05841318],
       [ 0.03775097, -0.05238622, -0.03534551, ...,  0.03876996,
         0.00456922, -0.03035389],
       [-0.02745359,  0.01689442, -0.01442918, ..., -0.00044589,
         0.01598724, -0.01762458],
       ...,
       [ 0.00382108,  0.03070448,  0.01528697, ...,  0.00110254,
         0.02784413,  0.0175535 ],
       [ 0.00428098, -0.04447004, -0.00806002, ..., -0.03650681,
        -0.04077066, -0.0274652 ],
       [ 0.00162171,  0.08250228, -0.00743312, ...,  0.00394707,
        -0.01803309, -0.00895644]], dtype=float32)

In [ ]:
embedding_test

array([[ 0.01120974, -0.03313988,  0.00473972, ..., -0.0256553 ,
        -0.05266365, -0.05841318],
       [ 0.03775097, -0.05238622, -0.03534551, ...,  0.03876996,
         0.00456922, -0.03035389],
       [-0.02745359,  0.01689442, -0.01442918, ..., -0.00044589,
         0.01598724, -0.01762458],
       ...,
       [ 0.00382108,  0.03070448,  0.01528697, ...,  0.00110254,
         0.02784413,  0.0175535 ],
       [ 0.00428098, -0.04447004, -0.00806002, ..., -0.03650681,
        -0.04077066, -0.0274652 ],
       [ 0.00162171,  0.08250228, -0.00743312, ...,  0.00394707,
        -0.01803309, -0.00895644]], dtype=float32)

In [ ]:
atrain = np.array(embedding_train)
atest = np.array(embedding_test)
btrain = np.array(X_train.toarray())
btest = np.array(X_test)

In [ ]:
Xtrain = [np.append(list1, list2) for list1, list2 in zip(embedding_train, X_train.toarray())]
Xtest = [np.append(list1, list2) for list1, list2 in zip(embedding_test, X_test.toarray())]

In [ ]:
ytrain = train["category"].tolist()
ytest = test["category"].tolist()

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='rbf', C=1, verbose=True)
svm_model.fit(Xtrain, ytrain)
ypred = svm_model.predict(Xtest)
print(classification_report(ytest, ypred))

[LibSVM]              precision    recall  f1-score   support

           0       0.73      0.97      0.83       917
           1       0.67      0.10      0.17        40
           2       0.48      0.25      0.33        95
           3       0.75      0.50      0.60       218
           4       0.75      0.14      0.24        43
           5       0.25      0.02      0.04        53
           6       0.89      0.56      0.68        70
           7       1.00      0.16      0.28        50

    accuracy                           0.73      1486
   macro avg       0.69      0.34      0.40      1486
weighted avg       0.71      0.73      0.68      1486



In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(Xtrain, ytrain)
ypred = mlp.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83       917
           1       0.24      0.20      0.22        40
           2       0.49      0.35      0.40        95
           3       0.60      0.57      0.58       218
           4       0.47      0.42      0.44        43
           5       0.42      0.38      0.40        53
           6       0.59      0.63      0.61        70
           7       0.38      0.38      0.38        50

    accuracy                           0.71      1486
   macro avg       0.50      0.47      0.48      1486
weighted avg       0.69      0.71      0.70      1486



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_classifier = RandomForestClassifier(n_estimators=1000 ,verbose=True, n_jobs=-1, oob_score = True)
RF_classifier.fit(Xtrain, ytrain)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  3.0min finished


RandomForestClassifier(n_estimators=1000, n_jobs=-1, oob_score=True,
                       verbose=True)

In [ ]:
ypred = RF_classifier.predict(Xtest)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.7s finished


In [ ]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.66      0.99      0.79       917
           1       0.00      0.00      0.00        40
           2       0.36      0.14      0.20        95
           3       0.73      0.18      0.29       218
           4       0.00      0.00      0.00        43
           5       0.00      0.00      0.00        53
           6       1.00      0.26      0.41        70
           7       1.00      0.10      0.18        50

    accuracy                           0.66      1486
   macro avg       0.47      0.21      0.23      1486
weighted avg       0.62      0.66      0.57      1486



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
result = pd.read_csv('ta-en-misogyny-test.csv',sep='\t', names = ["text"])
result = result.dropna()
result

,text
0,ante peddi yedukkure ponnai vacci panam sambat...
1,Yella idangalilum pengalai asinga paduthuvathe...
2,Pannikku porandha paradhesi H Raja... Ponna pu...
3,Ayya white T-shirt! Tamil culture pathi mulusa...
4,Madu vandhi bjb fraudulent lady
...,...
1852,Avanga sonanga pari 8:50 sema
1853,Bro Avanga great bro semma character antha akk...
1854,NAGENDRAN NAGA hii Ur watsapp number
1855,Nengal solvathu sari endral angalum kuda vethi...


In [ ]:
result['text']= result['text'].apply(lambda x:tolower(x))
result['text']= result['text'].apply(lambda x:removepunctuation(x))
result['text']= result['text'].apply(lambda x:deEmojify(x))
result['text']= result['text'].apply(lambda x:removestopwords(x))

In [ ]:
result

,text
0,ante peddi yedukkure ponnai vacci panam sambat...
1,yella idangalilum pengalai asinga paduthuvathe...
2,pannikku porandha paradhesi raja ponna pundaiy...
3,ayya white tshirt tamil culture pathi mulusa p...
4,madu vandhi bjb fraudulent lady
...,...
1852,sonanga pari 850 sema
1853,great semma character antha akka avangalukku v...
1854,nagendran naga hii watsapp number
1855,nengal solvathu sari endral angalum kuda vethi...


In [ ]:
embedding_pred = embedding_model.encode(result["text"].tolist())

In [ ]:
x_pred = tfidfvectorizer.fit_transform(result["text"])

In [ ]:
Xresult = [np.append(list1, list2) for list1, list2 in zip(embedding_pred, x_pred.toarray())]

In [ ]:
y_pred = svm_model.predict(Xtest)

In [ ]:
a = pd.DataFrame(y_pred.tolist(), columns = ["result"])

In [ ]:
a

,result
0,0
1,0
2,0
3,6
4,0
...,...
1481,1
1482,0
1483,0
1484,0


In [ ]:
key = { 0: 'None-of-the-above', 1:'Transphobic', 2: 'Counter-speech', 3:'Misandry',
       4:'Homophobia', 5:'Hope-Speech', 6:'Xenophobia', 7:'Misogyny'}

In [ ]:
a["label"] = a["result"].map(key)

In [ ]:
a


,result,label
0,0,None-of-the-above
1,0,None-of-the-above
2,0,None-of-the-above
3,6,Xenophobia
4,0,None-of-the-above
...,...,...
1481,1,Transphobic
1482,0,None-of-the-above
1483,0,None-of-the-above
1484,0,None-of-the-above


In [ ]:
final = a[["label"]]

In [ ]:
final

,label
0,None-of-the-above
1,None-of-the-above
2,None-of-the-above
3,Xenophobia
4,None-of-the-above
...,...
1481,Transphobic
1482,None-of-the-above
1483,None-of-the-above
1484,None-of-the-above


In [ ]:
list_of_tuples = list(zip(result["text"].tolist(), a["label"].tolist()))

In [ ]:
finalresult = pd.DataFrame(list_of_tuples,
                  columns = ['test_id', 'class_label'])

In [ ]:
finalresult

,test_id,class_label
0,ante peddi yedukkure ponnai vacci panam sambat...,None-of-the-above
1,yella idangalilum pengalai asinga paduthuvathe...,None-of-the-above
2,pannikku porandha paradhesi raja ponna pundaiy...,None-of-the-above
3,ayya white tshirt tamil culture pathi mulusa p...,Xenophobia
4,madu vandhi bjb fraudulent lady,None-of-the-above
...,...,...
1481,itha vida next level virus onnu disease x,Transphobic
1482,pranesh g video nala paruga avaga kai vachathu...,None-of-the-above
1483,mobil sand friend,None-of-the-above
1484,pu bg thevdiya payalr appanuku thaan poranthen...,None-of-the-above


In [ ]:
finalresult.to_csv("pandas_ta-en_run2.tsv", sep='\t', header = None)